In [7]:
import torch
from torchvision import models, transforms
from PIL import Image

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 5

# Define class names (should match the ones used in training)
class_names = [
    'Acne and Rosacea Photos', 
    'Eczema Photos', 
    'Psoriasis pictures', 
    'Urticaria Hives', 
    'Herpes, HPV, and other STDs'
]

def load_model(model_path):
    # Initialize the EfficientNet-B0 model
    model = models.efficientnet_b0(pretrained=False)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = torch.nn.Linear(num_ftrs, num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

def predict_image(image_path, model):
    # Define the preprocessing transforms (must match the training transforms)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    # Open and preprocess the image
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0).to(device)
    
    # Get prediction and confidence from the model
    with torch.no_grad():
        outputs = model(input_batch)
        # Compute probabilities using softmax
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        confidence, preds = torch.max(probabilities, 1)
    
    predicted_class = class_names[preds.item()]
    confidence_percentage = confidence.item() * 100  # Convert to percentage
    
    return predicted_class, confidence_percentage



In [9]:

if __name__ == '__main__':
    # Provide the path to your saved best model
    model_path = '/kaggle/input/try/tensorflow2/default/1/model_epoch_9.pth'
    model = load_model(model_path)
    
    # Provide the path to your test image
    test_image_path = '/kaggle/input/dermnet/test/Acne and Rosacea Photos/07VascularFace0120.jpg'
    predicted_class, confidence = predict_image(test_image_path, model)
    print(f"Prediction: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%")


<ipython-input-7-791ab5911580>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Prediction: Acne and Rosacea Photos
Confidence: 84.16%
